In [6]:
#Set these
f1 = "./../../taiyakiOutputs/output_createfasta.hdf5"
f2 = "./../../taiyakiOutputs/output_justfromfasta.hdf5"

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import h5py
import numpy as np
import statistics 

In [3]:
def get_reads_dict(filename):
    file = h5py.File(filename, "r")
    file = file['Reads']
    reads = []
    for r in file.keys():
        elem = {}
        elem['UUID'] = r
        for k in file[r].keys():
            elem[k]=file[r][k][()]
        reads.append(elem)
    return reads

def normalise_list(lst):
    mmin = min(lst)
    mmax = max(lst)
    lst = (lst - mmin)/(mmax - mmin)
    return lst

def are_signals_equal(s1,s2):
    if(len(s1) != len(s2)):
        return False

    are_equal = True
    for idx, val in enumerate(s1):
        if(s1[idx] != s2[idx]):
            return False
    return True

def find_matches_in_tayiaki_ouput(mapped_reads, unmapped_reads):
    matched = []
    unmatched = []

    for r1 in mapped_reads:
        s1 = r1['Dacs']
        found = None
        for r2 in unmapped_reads:
            s2 = r2['Dacs']
            if(are_signals_equal(s1,s2)):
                found = r2
        if(found != None):
            matched.append([r1,found])
        else:
            unmatched.append(r1)
    
    return matched, unmatched

def load_reads(f1,f2):
    unmapped_reads = get_reads_dict(f1)
    mapped_reads = get_reads_dict(f2)
    
    return find_matches_in_tayiaki_ouput(mapped_reads, unmapped_reads)

def get_read_attributes(read):
    #id, signal, points, labels
    return read['UUID'], normalise_list(read['Dacs']), read['Ref_to_signal'], read['Reference']

In [ ]:
def get_matched_reads_percent(f1, f2):
    reads, other_reads = load_reads(f1,f2)
    return f"Mapping to reference caused the loss of {100 - (len(other_reads) / len(reads) * 100)}% of data"

print(get_matched_reads_percent(f1,f2))